In [269]:
import json
from tkinter import *
from math import *
from collections import deque
import networkx as nx
import copy
import random as rd
import copy
import numpy as np
import numpy.linalg
import time
from os import listdir

## Read and write json inputs/outputs

In [227]:
#json inputs only

def read_input_instance(file_name) : 
    
    with open('datasets/json/'+file_name+'.json') as json_data:
        
        instance = json.load(json_data)
        if ('width' not in instance) :
            instance['width'] = 10**6
        if ('height' not in instance) :
            instance['height'] = 10**6
            
    return instance

def read_output_instance(file_name) : 
    
    with open('outputs/json/'+file_name+'.json') as json_data:
        
        instance = json.load(json_data)
        if ('width' not in instance) :
            instance['width'] = 10**6
        if ('height' not in instance) :
            instance['height'] = 10**6
            
    return instance

def write_instance(instance, file_name) : 
    
    with open('outputs/json/'+file_name+'.json', 'w') as json_data:
        json.dump(instance, json_data)
        

## Auxiliary functions to extract information from the graph

In [228]:
#Gives a list of all nodes (index=id since all of our instances are in the right order)
def get_nodes(instance) :    
    N = []
    for node in instance['nodes'] :
        N.append((node['x'],node['y']))
    return N


#Gives the value (x,y) to the node of id i
def set_node(instance, i, x, y):
    instance['nodes'][i] = {'id': i, 'x': x, 'y': y}
    

#Gives a list of all bends
def get_bends(instance) :     
    B = []
    for edge in instance['edges'] : 
        if 'bends' in edge :
            for bend in edge['bends'] :
                B.append((bend['x'],bend['y']))
    return B


#Gives a list of all straight line edges (if an edge is bended, there will be two elements)
def get_edges(instance) :
    
    node = get_nodes(instance)
    E = []
    for edge in instance['edges'] :
        
        if 'bends' in edge :

            n = len(edge['bends'])

            i1,j1 = node[edge['source']]
            i2,j2 = edge['bends'][0]['x'],edge['bends'][0]['y']
            E.append((i1,j1,i2,j2))

            i1,j1 = node[edge['target']]
            i2,j2 = edge['bends'][-1]['x'],edge['bends'][-1]['y']
            E.append((i1,j1,i2,j2))

            for i in range (1,n): 

                i1,j1 = edge['bends'][i-1]['x'],edge['bends'][i-1]['y']
                i2,j2 = edge['bends'][i]['x'],edge['bends'][i]['y']
                E.append((i1,j1,i2,j2))

        else :

            i1,j1 = node[edge['source']]
            i2,j2 = node[edge['target']]
            E.append((i1,j1,i2,j2))
            
    return E


#Gives at index i the list of all the neighbors of vertex i
def get_neighbors(instance):
    
    n = len(instance['nodes'])
    
    N = n*[[]]

    for edge in instance['edges'] :
        s = edge['source']
        t = edge['target']
        N[s] = N[s] + [t]
        N[t] = N[t] + [s]
        
    return N


#Takes the dictionnary format of an edge and find its length (takes bendings into account)
def get_edge_length(edge,instance) :
    
    if 'bends' in edge :
        nb_bends = len(edge['bends'])
    else :
        nb_bends = 0
        
    
    if nb_bends == 0 :
        
        return sqrt( (instance['nodes'][edge['source']]['x']-instance['nodes'][edge['target']]['x'])**2 + (instance['nodes'][edge['source']]['y']-instance['nodes'][edge['target']]['y'])**2 )
    
    else : 
        
        length = 0
        
        for i in range (nb_bends -1) :
            length += sqrt( (edge['bends'][i]['x']-edge['bends'][i+1]['x'])**2 + (edge['bends'][i]['y']-edge['bends'][i+1]['y'])**2 )
            
        length += sqrt( (instance['nodes'][edge['source']]['x']-edge['bends'][0]['x'])**2 + (instance['nodes'][edge['source']]['y']-edge['bends'][0]['y'])**2 )
        length += sqrt( (instance['nodes'][edge['target']]['x']-edge['bends'][nb_bends-1]['x'])**2 + (instance['nodes'][edge['target']]['y']-edge['bends'][nb_bends-1]['y'])**2 )
        
        return length
    

#Gives a list of the length of each edge
def get_edges_length(instance) : 
    return [get_edge_length(edge,instance) for edge in instance['edges']]


#Checks the planarity of a graph with networkx library
def is_planar (instance) : 
    
    G1 = nx.Graph()
    n = len(instance['nodes'])
    nodes = [i for i in range (0,n)]
    G1.add_nodes_from(nodes)
    for edge in instance['edges'] :
        G1.add_edge(edge['source'],edge['target'])

    G2 = nx.algorithms.planarity.check_planarity(G1)
        
    return G2[0]


#Returns the indices of the nodes from the closest to the furthest from the center
def sort_nodes(instance,center) : 
    
    nodes_and_distances = []
    
    for i in range (len(instance['nodes'])) : 
        
        p = (instance['nodes'][i]['x'],instance['nodes'][i]['y'])
        distance = norm2d_diff(p,center)
        
        j = 0
        while (j<len(nodes_and_distances) and distance>nodes_and_distances[j][1]) : 
            j += 1
        nodes_and_distances = nodes_and_distances[:j]+[(i,distance)]+nodes_and_distances[j:]
    
    sorted_nodes = [nodes_and_distances[i][0] for i in range (len(nodes_and_distances))]
    return sorted_nodes


#Returns the maximum number of vertices into the same grid case (graph with float coordinates)
def max_case(instance) : 
    
    grid = {}
    for x in range (instance['width']) :
        for y in range (instance['height']) : 
            grid[(x,y)] = 0 
            
    for node in instance['nodes'] : 
        x = floor(node['x'])
        y = floor(node['y'])
        if x == instance['width'] :
            x -= 1
        if y == instance['height'] :
            y -= 1
        grid[(x,y)] += 1
        
    nb_max = 0
    
    for x in range (instance['width']-1) :
        for y in range (instance['height']-1) : 
            if grid[(x,y)] > nb_max : 
                nb_max = grid[(x,y)]
                
    return nb_max


#Removes a list of nodes from an instance but does not re-index the nodes
def cut_out(instance,nodes) : 
    
    new_instance = {'nodes':[],'edges':[],'height':instance['height'],'width':instance['width'],'bends':instance['bends']}
    
    for node in instance['nodes'] :
        if node['id'] not in nodes :
            new_instance['nodes'].append(copy.copy(node))
    for edge in instance['edges'] :
        if (edge['source'] not in nodes) and (edge['target'] not in nodes) :
            new_instance['edges'].append(copy.copy(edge))
        
    return new_instance


def get_depth (instance,outer_face) : 

    depth = 0
    
    neighbors = {}
    for node in instance['nodes'] : 
        neighbors[node['id']] = []
    for edge in instance['edges'] : 
        if edge['target'] not in neighbors[edge['source']] : 
            neighbors[edge['source']].append(edge['target'])
        if edge['source'] not in neighbors[edge['target']] : 
            neighbors[edge['target']].append(edge['source'])
            
    depth_board = {}
    reversed_depth_board = {0:[]}
    
    while (len(instance['nodes']) > 0) : 
        next_neighbors = []
        for i in outer_face : 
            for j in neighbors[i] :
                if (j not in next_neighbors) and (j not in outer_face) : 
                    next_neighbors.append(j)
                    
        for i in outer_face : 
            depth_board[i] = depth
            reversed_depth_board[depth].append(i)
            
        instance = cut_out(instance,outer_face)
        outer_face = next_neighbors
        depth += 1
        reversed_depth_board[depth] = []
        
        neighbors = {}
        for node in instance['nodes'] : 
            neighbors[node['id']] = []
        for edge in instance['edges'] : 
            if edge['target'] not in neighbors[edge['source']] : 
                neighbors[edge['source']].append(edge['target'])
            if edge['source'] not in neighbors[edge['target']] : 
                neighbors[edge['target']].append(edge['source'])
                
    return depth_board, reversed_depth_board, depth-1


#Puts the bends into the nodes and save their new id
def get_bend_correspondance(instance) :
    
    n = len(instance['nodes'])
    bend_instance = {'nodes':copy.deepcopy(instance['nodes']),'edges':[],'height':instance['height'],'width':instance['width'],'bends':instance['bends']}
    
    #an bend is represented by the index of its edge and its order among the other bends
    correspondance = {}
    nb_bends = 0
    for i in range (len(instance['edges'])) :
        edge = instance['edges'][i]
        if 'bends' in edge :
            for j in range (len(edge['bends'])) :
                correspondance[n+nb_bends] = (i,j)
                bend_instance['nodes'].append({'id':n+nb_bends,'x':edge['bends'][j]['x'],'y':edge['bends'][j]['y']})
                if (j==0 and j<len(edge['bends'])-1) : 
                    bend_instance['edges'].append({'source':edge['source'],'target':n+nb_bends})
                    bend_instance['edges'].append({'source':n+nb_bends,'target':n+nb_bends+1})
                elif (j==0 and j==len(edge['bends'])-1) :
                    bend_instance['edges'].append({'source':edge['source'],'target':n+nb_bends})
                    bend_instance['edges'].append({'source':n+nb_bends,'target':edge['target']})
                    
                elif (j==len(edge['bends'])-1) :
                    bend_instance['edges'].append({'source':n+nb_bends,'target':edge['target']})              
                else :
                    bend_instance['edges'].append({'source':n+nb_bends,'target':n+nb_bends+1})
                    
                nb_bends += 1
                
        else :
            bend_instance['edges'].append(edge)
    return bend_instance,correspondance


#Reverse the operation by putting back the bends into the right edge
def apply_bend_correspondance(instance,bend_instance,correspondance) : 
    
    for node in bend_instance['nodes'] : 
            if (node['id'] < len(instance['nodes'])) : 
                instance['nodes'][node['id']]['x'] = bend_instance['nodes'][node['id']]['x']
                instance['nodes'][node['id']]['y'] = bend_instance['nodes'][node['id']]['y']
            else : 
                i,j = correspondance[node['id']]
                instance['edges'][i]['bends'][j]['x'] = bend_instance['nodes'][node['id']]['x']
                instance['edges'][i]['bends'][j]['y'] = bend_instance['nodes'][node['id']]['y']
                
    return instance
        

## Auxiliary geometry functions 

In [229]:
def norm2d (p) : 
    return sqrt(p[0]**2+p[1]**2)

def triangle_area(p1,p2,p3) :

    l1 = norm2d((p2[0]-p1[0],p2[1]-p1[1]))
    l2 = norm2d((p3[0]-p2[0],p3[1]-p2[1]))
    l3 = norm2d((p1[0]-p3[0],p1[1]-p3[1]))
    p = (l1+l2+l3)/2.
    if (p*(p-l1)*(p-l2)*(p-l3) < 0) :
        return 0
    S = sqrt(p*(p-l1)*(p-l2)*(p-l3))

    return S

def is_in_triangle (p,p1,p2,p3):
    
    lambda1 = triangle_area(p,p2,p3)
    lambda2 = triangle_area(p,p1,p3)
    lambda3 = triangle_area(p,p1,p2)
    
    total = lambda1 + lambda2 + lambda3
    
    lambda1 /= total
    lambda2 /= total
    lambda3 /= total
    
    interior_p = (lambda1*p1[0]+lambda2*p2[0]+lambda3*p3[0],lambda1*p1[1]+lambda2*p2[1]+lambda3*p3[1])
    epsilon = 10**(-3)
    
    return (abs(p[0]-interior_p[0]) < epsilon) and (abs(p[1]-interior_p[1]) < epsilon)


# p1.p2
def dot (p1,p2) : 
    return p1[0]*p2[0] + p1[1]*p2[1]


# || p1-p2 ||
def norm2d_diff (p1,p2) : 
    return sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)


# p1-p2
def diff(p1,p2) :
    return (p1[0]-p2[0],p1[1]-p2[1])


#Assumes the outer face is convex
def get_outer_face (instance) :
    
    nodes = get_nodes(instance)
    neighbors = get_neighbors(instance)
    
    #find the lowest point
    
    index_min = 0
    xmin = instance['nodes'][0]['x']
    
    for i in range (1,len(instance['nodes'])) :
        x = instance['nodes'][i]['x']
        if (x < xmin) : 
            index_min = i
            xmin = x
    
    outer_face = [index_min]
    
    #iteratively find the next point of the outer face
    next_index = neighbors[index_min][0]
    cosinus = dot(diff(nodes[next_index],nodes[index_min]),(1,0))/norm2d_diff(nodes[next_index],nodes[index_min])
    if cosinus < -1 :
        cosinus = -1
    if cosinus > 1 :
        cosinus = 1
    angle_min = acos(cosinus)
    for v in neighbors[index_min] :
        cosinus = dot(diff(nodes[v],nodes[index_min]),(1,0))/norm2d_diff(nodes[v],nodes[index_min])
        if cosinus < -1 :
            cosinus = -1
        if cosinus > 1 :
            cosinus = 1
        angle = acos(cosinus)
        if (angle < angle_min) :
            next_index = v
            angle_min = angle
            
    outer_face.append(next_index)
    index_min = next_index
    
    while (index_min != outer_face[0]) : 
        
        next_index = neighbors[index_min][0]
        cosinus = dot(diff(nodes[next_index],nodes[index_min]),diff(nodes[outer_face[-1]],nodes[outer_face[-2]]))/(norm2d_diff(nodes[next_index],nodes[index_min])*norm2d_diff(nodes[outer_face[-1]],nodes[outer_face[-2]]))
        if cosinus < -1 :
            cosinus = -1
        if cosinus > 1 :
            cosinus = 1
        angle_min = acos(cosinus)
        for v in neighbors[index_min] :
            cosinus = dot(diff(nodes[v],nodes[index_min]),diff(nodes[outer_face[-1]],nodes[outer_face[-2]]))/(norm2d_diff(nodes[v],nodes[index_min])*norm2d_diff(nodes[outer_face[-1]],nodes[outer_face[-2]]))
            if cosinus < -1 :
                cosinus = -1
            if cosinus > 1 :
                cosinus = 1
            angle = acos(cosinus)
            if (angle < angle_min) :
                next_index = v
                angle_min = angle
        
        outer_face.append(next_index)
        index_min = next_index
        
    return outer_face[:-1]


def get_convex_hull (instance) :
    
    epsilon = 10**(-3)
    
    nodes = get_nodes(instance)
    if len(nodes) <= 1 :
        return [i for i in range (len(nodes))]
    
    #find the lowest point
    
    index_min = 0
    xmin = instance['nodes'][0]['x']
    
    for i in range (1,len(instance['nodes'])) :
        x = instance['nodes'][i]['x']
        if (x < xmin) : 
            index_min = i
            xmin = x
    
    hull = [index_min]
    
    #iteratively find the next point of the convex hull
    if (index_min != 0) : 
        next_index = 0
    else : 
        next_index = 1
        
    node1 = nodes[index_min]
    node2 = nodes[next_index]
    cosinus = dot(diff(node2,node1),(1,0))/norm2d_diff(node2,node1)
    if cosinus < -1 :
        cosinus = -1
    if cosinus > 1 :
        cosinus = 1
    angle_min = acos(cosinus)
    distance_min = norm2d_diff(node2,node1)
    
    for i in range (len(nodes)) :
        if i != index_min : 
            node1 = nodes[index_min]
            node2 = nodes[i]
            cosinus = dot(diff(node2,node1),(1,0))/norm2d_diff(node2,node1)
            if cosinus < -1 :
                cosinus = -1
            if cosinus > 1 :
                cosinus = 1
            angle = acos(cosinus)
            if (abs(angle-angle_min) < epsilon):
                if (norm2d_diff(node2,node1) < distance_min) : 
                    next_index = i
                    angle_min = angle
                    distance_min = norm2d_diff(node2,node1)
            elif (angle < angle_min - epsilon):
                next_index = i
                angle_min = angle
                distance_min = norm2d_diff(node2,node1)
            
    hull.append(next_index)
    index_min = next_index
    
    while (index_min != hull[0]) : 
        
        if (index_min != 0) : 
            next_index = 0
        else : 
            next_index = 1
            
        node1 = nodes[index_min]
        node2 = nodes[next_index]
        node3 = nodes[hull[-2]]
        node4 = nodes[hull[-1]]
        cosinus = dot(diff(node2,node1),diff(node4,node3))/(norm2d_diff(node2,node1)*norm2d_diff(node4,node3))
        if cosinus < -1 :
            cosinus = -1
        if cosinus > 1 :
            cosinus = 1
        angle_min = acos(cosinus)
        
        for i in range (len(nodes)) :
            if i != index_min : 
                node1 = nodes[index_min]
                node2 = nodes[i]
                node3 = nodes[hull[-2]]
                node4 = nodes[hull[-1]]
                cosinus = dot(diff(node2,node1),diff(node4,node3))/(norm2d_diff(node2,node1)*norm2d_diff(node4,node3))
                if cosinus < -1 :
                    cosinus = -1
                if cosinus > 1 :
                    cosinus = 1
                angle = acos(cosinus)
                if (abs(angle-angle_min) < epsilon):
                    if (norm2d_diff(node2,node1) < distance_min) : 
                        next_index = i
                        angle_min = angle
                        distance_min = norm2d_diff(node2,node1)
                elif (angle < angle_min-epsilon):
                    next_index = i
                    angle_min = angle
                    distance_min = norm2d_diff(node2,node1)
        
        hull.append(next_index)
        index_min = next_index
        
    return hull[:-1]


## Checking output

In [332]:
#Checks if an instance is valid

def is_valid(instance, verbose=False, integer_coordinates=True) :
    
    #empty graph
    if 'nodes' not in instance : 
        return False
    
    epsilon = 10**(-2)
    
    node = get_nodes(instance)
    bend = get_bends(instance)
    vertices = node+bend
    
    xmin, xmax = min(vertices, key = lambda t:t[0])[0], max(vertices, key = lambda t:t[0])[0]
    ymin, ymax = min(vertices, key = lambda t:t[1])[1], max(vertices, key = lambda t:t[1])[1]
    
    #Check if it does not go beyond maximum grid size
    if (xmax-xmin > instance['width']):
        if verbose :
            print('width')
        return False
    
    if (ymax-ymin > instance['height']):
        if verbose : 
            print('height')
        return False
    
    seen_pos = {}
    for vertex in vertices :
        
        #Check if there are not two vertices on top of each other
        if vertex in seen_pos:
            if verbose:
                print('superimposed points')
            return False
        seen_pos[vertex] = 1
        
        #Check if coordinates are integers
        if integer_coordinates : 
            
            if (vertex[0]-floor(vertex[0]) > 10**(-6)) :
                if verbose :
                    print(f'x not integer : {vertex}')
                return False
            if (vertex[1]-floor(vertex[1]) > 10**(-6)) :
                if verbose :
                    print(f'y not integer : {vertex}')
                return False
        
    E = get_edges(instance)
    
    for i in range (len(E)) :        
        for j in range (i+1,len(E)):

            edge1 = E[i]
            edge2 = E[j]

            #both segments have length > 0
            if (abs(edge1[2]-edge1[0]) > epsilon or abs(edge1[3]-edge1[1]) > epsilon) and (abs(edge2[2]-edge2[0]) > epsilon or abs(edge2[3]-edge2[1]) > epsilon) :
               
                #both equations are x = constant
                if (abs(edge1[2]-edge1[0]) <= epsilon) and (abs(edge2[2]-edge2[0]) <= epsilon) :
                    
                    #same constant
                    if (abs(edge1[0]-edge2[0]) <= epsilon) :
                        
                        #check overlapping
                        x = edge2[0]
                        y = edge2[1]
                        t = (y-edge1[1])/(edge1[3]-edge1[1])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose :
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False

                        x = edge2[2]
                        y = edge2[3]
                        t = (y-edge1[1])/(edge1[3]-edge1[1])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose : 
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False
                        
                        x = edge1[0]
                        y = edge1[1]
                        t = (y-edge2[1])/(edge2[3]-edge2[1])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose :
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False

                        x = edge1[2]
                        y = edge1[3]
                        t = (y-edge2[1])/(edge2[3]-edge2[1])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose : 
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False
                        
                #first equation only is x = constant
                elif (abs(edge1[2]-edge1[0]) <= epsilon) :
                    
                    #Find second line affine parameters
                    a2 = (edge2[3]-edge2[1])/(edge2[2]-edge2[0])
                    b2 = edge2[1] - edge2[0]*a2
                    
                    #Compute intersection and find if it belongs to both segments
                    x = edge1[0]
                    y = a2*x + b2

                    t1 = (y-edge1[1])/(edge1[3]-edge1[1])
                    t2 = (x-edge2[0])/(edge2[2]-edge2[0])

                    if (epsilon<t1 and t1<1-epsilon and epsilon<t2 and t2<1-epsilon) :
                        if verbose :
                            print('intersection')
                            print(x,y,t1,t2)
                            print(edge1)
                            print(edge2)
                        return False
                    
                #second equation only is x = constant
                elif (abs(edge2[2]-edge2[0]) <= epsilon) :
                    
                    #Find first line affine parameters
                    a1 = (edge1[3]-edge1[1])/(edge1[2]-edge1[0])
                    b1 = edge1[1] - edge1[0]*a1
                    
                    #Compute intersection and find if it belongs to both segments
                    x = edge2[0]
                    y = a1*x + b1

                    t1 = (x-edge1[0])/(edge1[2]-edge1[0])
                    t2 = (y-edge2[1])/(edge2[3]-edge2[1])

                    if (epsilon<t1 and t1<1-epsilon and epsilon<t2 and t2<1-epsilon) :
                        if verbose :
                            print('intersection')
                            print(x,y,t1,t2)
                            print(edge1)
                            print(edge2)
                        return False
                    
                #both equations of type y = ax+b     
                else : 

                    #Find line affine parameters

                    a1 = (edge1[3]-edge1[1])/(edge1[2]-edge1[0])
                    b1 = edge1[1] - edge1[0]*a1

                    a2 = (edge2[3]-edge2[1])/(edge2[2]-edge2[0])
                    b2 = edge2[1] - edge2[0]*a2

                    #If lines are the same, check if there is some overlapping
                    if (abs(a2-a1)<epsilon and abs(b2-b1)<epsilon) :

                        x = edge2[0]
                        y = edge2[1]
                        t = (x-edge1[0])/(edge1[2]-edge1[0])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose :
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False

                        x = edge2[2]
                        y = edge2[3]
                        t = (x-edge1[0])/(edge1[2]-edge1[0])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose : 
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False
                        
                        x = edge1[0]
                        y = edge1[1]
                        t = (x-edge2[0])/(edge2[2]-edge2[0])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose :
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False

                        x = edge1[2]
                        y = edge1[3]
                        t = (x-edge2[0])/(edge2[2]-edge2[0])

                        if (epsilon<t and t<1-epsilon) :
                            if verbose : 
                                print('overlapping')
                                print(edge1)
                                print(edge2)
                            return False

                    #lines are not parallel
                    elif (abs(a2-a1) >= epsilon):

                        #Compute intersection and find if it belongs to both segments
                        x = (b1-b2)/(a2-a1)
                        y = a1*x + b1

                        t1 = (x-edge1[0])/(edge1[2]-edge1[0])
                        t2 = (x-edge2[0])/(edge2[2]-edge2[0])

                        if (epsilon<t1 and t1<1-epsilon and epsilon<t2 and t2<1-epsilon) :
                            if verbose :
                                print('intersection')
                                print(x,y,t1,t2)
                                print(edge1)
                                print(edge2)
                            return False

                    
    return True
                    

In [231]:
#If the instance is not valid and there is a length = 0, returns 10**9
def get_polyline_edge_length_ratio (instance, verbose = False) :
    
    max_length = 0
    min_length = sqrt(instance['height']**2 + instance['width']**2)
    
    for edge in instance['edges'] :
        length = get_edge_length(edge,instance)
        if (length > max_length) :
            max_length = length
            edge_max = edge
        if (length < min_length) :
            min_length = length
            edge_min = edge
            
    if verbose :
        print(f'max edge : {edge_max} = {max_length}, min edge : {edge_min} = {min_length}, ratio = {max_length/min_length}')
        
    if abs(min_length) < 10**(-3) :
         return 10**9
    
    return max_length/min_length

## Tutte solution

#### Functions wrt cycles and faces in a graph

In [232]:
# Detect cycle in an undirected graph using BFS
# Code inspired from https://www.geeksforgeeks.org/detect-cycle-in-an-undirected-graph-using-bfs/

def addEdge(adj: list, u, v):
    adj[u].append(v)
    adj[v].append(u)
    
def is_cycle(cycle, adj) : 
    
    for i in range (len(cycle)-1) :
        if (cycle[i+1] not in adj[cycle[i]]) : 
            return False
        
    if (cycle[len(cycle)-1] not in adj[cycle[0]]) : 
        return False
    else : 
        return True
        
#Checks if there are interconnexions between the cycle vertices
def is_interconnected (cycle,instance) : 
    
    neighbors = get_neighbors(instance)
    n = len(cycle)
    
    for i in range (1,n-1) : 
        v = cycle[i]
        for u in neighbors[v] :
            if (u in cycle) and (u != cycle[i-1]) and (u != cycle[i+1]) :
                return True
            
    v = cycle[0]
    for u in neighbors[v] :
        if (u in cycle) and (u != cycle[n-1]) and (u != cycle[1]) :
            return True
        
    v = cycle[n-1]
    for u in neighbors[v] :
        if (u in cycle) and (u != cycle[n-2]) and (u != cycle[0]) :
            return True
        
    return False


#Checks if G \ cycle is still connected
def is_face (cycle,instance) : 
    
    new_instance = {'nodes':[],'edges':[],'height':instance['height'],'width':instance['width'],'bends':instance['bends']}
    
    for node in instance['nodes'] :
        if node['id'] not in cycle :
            new_instance['nodes'].append(copy.copy(node))
    for edge in instance['edges'] :
        if (edge['source'] not in cycle) and (edge['target'] not in cycle) :
            new_instance['edges'].append(copy.copy(edge))
            
    correspondance = {}
    i = 0
    for node in new_instance['nodes'] :
        old_id = node['id']
        new_id = i
        correspondance[old_id] = new_id
        node['id'] = new_id
        i += 1
        
    for edge in new_instance['edges'] :
        edge['source'] = correspondance[edge['source']]
        edge['target'] = correspondance[edge['target']]
        
        
    neighbors = get_neighbors(new_instance)
    for i in range (len(neighbors)) : 
        if len(neighbors[i]) == 0 :
            return False
        
    return True

    
def isCyclicConnected(adj: list, s, V, visited: list, instance):
 
    # Set parent vertex for every vertex as -1.
    parent = [-1] * V
    
    #Set the list of cycles empty
    cycles = []
 
    # Create a queue for BFS
    q = deque()
 
    # Mark the current node as 
    # visited and enqueue it
    visited[s] = True
    q.append(s)
 
    while (len(q)>0):
 
        # Dequeue a vertex from queue and print it
        u = q.pop()
 
        # Get all adjacent vertices of the dequeued
        # vertex u. If a adjacent has not been visited,
        # then mark it visited and enqueue it. We also
        # mark parent so that parent is not considered
        # for cycle.
        for v in adj[u]:
            if not visited[v]:
                visited[v] = True
                q.append(v)
                parent[v] = u
            elif parent[u] != v:
                cycle = [v,u]
                next_vertex = parent[u]
                while (next_vertex!=v and next_vertex!=-1) :
                    cycle.append(next_vertex)
                    next_vertex = parent[next_vertex]
                if (is_cycle(cycle,adj) and is_face(cycle,instance)) : 
                    cycles.append(cycle)
 
    return cycles
 
def isCyclicDisconnected(adj: list, V, instance):
 
    # Mark all the vertices as not visited
    visited = [False] * V
 
    all_cycles = []
    for i in range(V):
        if (not visited[i]) :
            cycles = isCyclicConnected(adj, i, V, visited, instance)
            for cycle in cycles :
                if cycle not in all_cycles :
                    all_cycles.append(cycle)
    return all_cycles
 
#Detects all the cycles in a graph
def detect_cycle (instance):
    V = len(instance['nodes'])
    adj = [[] for i in range(V)]
    for edge in instance['edges'] :
        addEdge(adj, edge['source'], edge['target'])
 
    return isCyclicDisconnected(adj, V, instance)


def bigger_cycle(cycles) :
    
    if len(cycles)==0 :
        return []
    
    index = 0
    M = len(cycles[0])
    
    for i in range (1,len(cycles)) :
        if (len(cycles[i]) > M) :
            index = i
            M = len(cycles[i])
            
    return cycles[index]


def smaller_cycle(cycles) :
    
    if len(cycles)==0 :
        return []
    
    index = 0
    M = len(cycles[0])
    
    for i in range (1,len(cycles)) :
        if (len(cycles[i]) < M) :
            index = i
            M = len(cycles[i])
            
    return cycles[index]


#### All functions wrt polygon faces, exterior/interior points

In [233]:
#all auxiliary functions to detect if a point is exterior to a given outer face (which is a regular polygon)

#Takes the outer face polygon and check if the point is outside of it
def is_exterior(p,outer_face,instance) : 
    
    radius = min(instance['height'],instance['width'])/2.0
    polygon_center = (radius,radius)

    n = len(outer_face)
    
    for i in range (n-1) : 
        p1 = (instance['nodes'][outer_face[i]]['x'],instance['nodes'][outer_face[i]]['y'])
        p2 = (instance['nodes'][outer_face[i+1]]['x'],instance['nodes'][outer_face[i+1]]['y'])
        if (is_in_triangle(p,polygon_center,p1,p2)) :
            return False
        
    p1 = (instance['nodes'][outer_face[n-1]]['x'],instance['nodes'][outer_face[n-1]]['y'])
    p2 = (instance['nodes'][outer_face[0]]['x'],instance['nodes'][outer_face[0]]['y'])

    if (is_in_triangle(p,polygon_center,p1,p2)) :
        return False
    
    return True


#Puts the outer face vertices on a maximum size regular polygon
def update_polygon(outer_face,instance,approximate) : 

    n = len(outer_face)
    radius = min(instance['height'],instance['width'])/2.0
    
    for i in range (n) : 
        
        x= radius + radius*cos(i*2*pi/n)
        y= radius + radius*sin(i*2*pi/n)
        
        if approximate : 
            x = find_closest_grid_point(x,instance['width'])
            y = find_closest_grid_point(y,instance['height'])
        
        instance['nodes'][outer_face[i]]['x'] = x
        instance['nodes'][outer_face[i]]['y'] = y
        
    return instance


#Puts vertices external to the outer polygon inside of it
def update_external_vertices (external_vertices,instance,approximate) :
    
    radius = min(instance['height'],instance['width'])/2.0
    
    if approximate : 
        x_center = find_closest_grid_point (radius,instance['width']) 
        y_center = find_closest_grid_point (radius,instance['height'])
    else : 
        x_center = radius
        y_center = radius
    
    for v in external_vertices : 
        instance['nodes'][v]['x'] = x_center
        instance['nodes'][v]['y'] = y_center
        
    return instance


#### Tutte iterative version

In [234]:
#Approximate a float coordinate by an integer bounded in [0,width]
def find_closest_grid_point (x,width) :
    
    if (x-floor(x) <= floor(x)+1-x) :
        if floor(x) >= 0 :
            return floor(x)
        else :
            return floor(x)+1
    else :
        if floor(x)+1<=width :
            return floor(x)+1
        else:
            return floor(x)


#Define an outer face and put every vertex inside
def tutte_initialization (instance, outer_face = None, smallest_outer_face = True, approximate = False) :
    
    if outer_face == None :
    
        if smallest_outer_face : 
            #find the smallest cycle in the graph and put it on an outer face polygon
            cycles = detect_cycle(instance)
            outer_face = smaller_cycle(cycles)

        else : 
            #take the outer face already present in the initial drawing
            if is_valid(instance) : 
                outer_face = get_outer_face(instance)
            else :
                outer_face = get_outer_face(find_planar_drawing(instance))
            
    instance = update_polygon(outer_face,instance,True)
    
    #put all the external vertices inside the polygon
    
    external_vertices = []
    for i in range (len(instance['nodes'])) : 
        p = (instance['nodes'][i]['x'],instance['nodes'][i]['y'])
        if ((i not in outer_face) and (is_exterior(p,outer_face,instance))) : 
            external_vertices.append(i)
    
    instance = update_external_vertices(external_vertices,instance,approximate)
        
    return (outer_face,instance)


#we consider that every vertex is inside the outer face, 
#and we apply the barycentric algorithm on the inside vertices once
def tutte_step (outer_face, instance, approximate = False) : 
    
    neighbors = get_neighbors(instance)
    
    n = len(instance['nodes'])
    for i in range (n) :
        
        if i not in outer_face : 
            
            degree = len(neighbors[i])
            x = 0
            y = 0
            
            for j in neighbors[i] :
                x += instance['nodes'][j]['x']
                y += instance['nodes'][j]['y']
            
            if approximate : 
                x = find_closest_grid_point(x/degree,instance['width'])
                y = find_closest_grid_point(y/degree,instance['height'])
                
            else :
                x = x/degree
                y = y/degree
            
            instance['nodes'][i]['x'] = x
            instance['nodes'][i]['y'] = y
            
    return instance
                   
    
#Apply Tutte iterative method until convergence    
def tutte (instance, outer_face = None, smallest_outer_face = True, approximate = False) : 

    bend_instance, correspondance = get_bend_correspondance(instance)
    
    old_instance = copy.deepcopy(bend_instance)
    outer_face, bend_instance = tutte_initialization(bend_instance, outer_face, smallest_outer_face, approximate)
    i = 0
    
    while (old_instance != bend_instance and i<10**(9)) : 
        old_instance = copy.deepcopy(bend_instance)
        bend_instance = tutte_step(outer_face,bend_instance, approximate)
        i += 1
    
    instance = apply_bend_correspondance(instance,bend_instance,correspondance)
    return instance


#Print the ratio of the graph after applying Tutte iterative version, testing every possible cycle as outer face
def test_all_tuttes (instance, approximate = False) : 
    
    possible_outer_faces = detect_cycle(instance)
    
    for outer_face in possible_outer_faces :
        
        print(outer_face)
        tutte_instance = tutte(instance,outer_face,True,False)
        tutte_instance = center_and_scale(tutte_instance,outer_face)
        if is_valid(tutte_instance,False,False) :
            print(f'ratio : {get_polyline_edge_length_ratio (tutte_instance)}')
            

#Choose the valid instance with minimum ratio among all possible outer faces
#Returns empty instance if not found
def best_tutte (instance) : 
    
    possible_outer_faces = detect_cycle(instance)
    
    best_ratio = 10**9
    best_instance = {}
    best_outer_face = []
    
    for outer_face in possible_outer_faces :
        tutte_instance = tutte(instance,outer_face)
        if is_valid(tutte_instance,False,False) :
            ratio = get_polyline_edge_length_ratio (tutte_instance)
            if ratio < best_ratio : 
                best_ratio = ratio
                best_instance = copy.deepcopy(tutte_instance)
                best_outer_face = outer_face
    
    return best_instance, best_outer_face
            

#### Tutte solver version

In [235]:
#Transforms the instance centered in (0,0) and scaled between -1 and 1
#into a maximum size graph
def center_and_scale(instance,outer_face) :
    
    coordinates = {}
    center = (0,0)
    n = len(outer_face)  
    epsilon = 10**(-3)
    
    bend_instance, correspondance = get_bend_correspondance(instance)
    
    #find the barycentric coordinates of the interior points according to the outer face
    
    for node in bend_instance['nodes'] : 
        if node['id'] not in outer_face : 
            
            for i in range (n) : 
                
                if i!=n-1 : 
                
                    p1 = (bend_instance['nodes'][outer_face[i]]['x'],bend_instance['nodes'][outer_face[i]]['y'])
                    p2 = (bend_instance['nodes'][outer_face[i+1]]['x'],bend_instance['nodes'][outer_face[i+1]]['y'])
                    p3 = center
                    p = (node['x'],node['y'])

                    lambda1 = triangle_area(p,p2,p3)
                    lambda2 = triangle_area(p,p1,p3)
                    lambda3 = triangle_area(p,p1,p2)
                    total = lambda1 + lambda2 + lambda3
                    lambda1 /= total
                    lambda2 /= total
                    lambda3 /= total

                    barycentric_p = (lambda1*p1[0]+lambda2*p2[0]+lambda3*p3[0],lambda1*p1[1]+lambda2*p2[1]+lambda3*p3[1])
                    if (norm2d_diff(p,barycentric_p) < epsilon) : 
                        node_coordinates = [0 for i in range (len(outer_face)+1)]
                        node_coordinates[i] = lambda1
                        node_coordinates[i+1] = lambda2
                        node_coordinates[-1] = lambda3
                        coordinates[node['id']] = node_coordinates
                        
                else : 
                    
                    p1 = (bend_instance['nodes'][outer_face[n-1]]['x'],bend_instance['nodes'][outer_face[n-1]]['y'])
                    p2 = (bend_instance['nodes'][outer_face[0]]['x'],bend_instance['nodes'][outer_face[0]]['y'])
                    p3 = center
                    p = (node['x'],node['y'])

                    lambda1 = triangle_area(p,p2,p3)
                    lambda2 = triangle_area(p,p1,p3)
                    lambda3 = triangle_area(p,p1,p2)
                    total = lambda1 + lambda2 + lambda3
                    lambda1 /= total
                    lambda2 /= total
                    lambda3 /= total

                    barycentric_p = (lambda1*p1[0]+lambda2*p2[0]+lambda3*p3[0],lambda1*p1[1]+lambda2*p2[1]+lambda3*p3[1])
                    if (norm2d_diff(p,barycentric_p) < epsilon) : 
                        node_coordinates = [0 for i in range (len(outer_face)+1)]
                        node_coordinates[n-1] = lambda1
                        node_coordinates[0] = lambda2
                        node_coordinates[-1] = lambda3
                        coordinates[node['id']] = node_coordinates
    
    #update polygon
                    
    radius = min(bend_instance['height'],bend_instance['width'])/2.0
    for i in range (n) : 
        
        x= radius + radius*cos(-i*2*pi/n)
        y= radius + radius*sin(-i*2*pi/n)    
        bend_instance['nodes'][outer_face[i]]['x'] = x
        bend_instance['nodes'][outer_face[i]]['y'] = y
        
    center = (bend_instance['width']/2,bend_instance['height']/2)
    
    #update inner points 
    
    for node in bend_instance['nodes'] : 
        if node['id'] not in outer_face : 
            
            bend_instance['nodes'][node['id']]['x'] = 0
            bend_instance['nodes'][node['id']]['y'] = 0
            
            for i in range (n) : 
                bend_instance['nodes'][node['id']]['x'] += coordinates[node['id']][i]*bend_instance['nodes'][outer_face[i]]['x']
                bend_instance['nodes'][node['id']]['y'] += coordinates[node['id']][i]*bend_instance['nodes'][outer_face[i]]['y']
            
            bend_instance['nodes'][node['id']]['x'] += coordinates[node['id']][-1]*center[0]
            bend_instance['nodes'][node['id']]['y'] += coordinates[node['id']][-1]*center[1]
    
    instance = apply_bend_correspondance(instance,bend_instance,correspondance)
    
    return instance


#Code inspired from : https://github.com/jadenstock/Tutte-embedding/blob/master/TutteEmbedding.py
 
def tutte_embedding(instance, outer_face=None):
    
    bend_instance,correspondance = get_bend_correspondance(instance)
    
    if outer_face == None :
        cycles = detect_cycle(bend_instance)
        outer_face = smaller_cycle(cycles)
        
    pos = {} #a dictionary of node positions
    tmp = nx.Graph()
    for i in range (len(outer_face)-1) :
        tmp.add_edge(outer_face[i],outer_face[i+1])
    tmp.add_edge(outer_face[len(outer_face)-1],outer_face[0])
    tmp_pos = nx.spectral_layout(tmp) #ensures that outerface is a convex shape
    pos.update(tmp_pos)
    outer_vertices = tmp.nodes()
    remaining_vertices = [node['id'] for node in bend_instance['nodes'] if node['id'] not in outer_vertices]

    size = len(remaining_vertices)
    A = [[0 for i in range(size)] for i in range(size)] #create the the system of equations that will determine the x and y positions of remaining vertices
    b = [0 for i in range(size)] #the elements of theses matrices are indexed by the remaining_vertices list
    C = [[0 for i in range(size)] for i in range(size)]
    d = [0 for i in range(size)]
    for u in remaining_vertices :
        i = remaining_vertices.index(u)
        neighbors = get_neighbors(bend_instance)[u]
        n = len(neighbors)
        A[i][i] = 1
        C[i][i] = 1
        for v in neighbors:
            if v in outer_vertices:
                b[i] += float(pos[v][0])/n
                d[i] += float(pos[v][1])/n
            else:
                j = remaining_vertices.index(v)
                A[i][j] = -(1/float(n))
                C[i][j] = -(1/float(n))

    x = np.linalg.solve(A, b)
    y = np.linalg.solve(C, d)
    for u in remaining_vertices:
        i = remaining_vertices.index(u)
        pos[u] = [x[i],y[i]]
        
        
    for node in pos :
        bend_instance['nodes'][node]['x'] = pos[node][0]
        bend_instance['nodes'][node]['y'] = pos[node][1]
        
    instance = apply_bend_correspondance(instance,bend_instance,correspondance)
    instance = center_and_scale(instance,outer_face)

    return instance
        

#Print the ratio of the graph after applying Tutte solver version, testing every possible cycle as outer face
def test_all_tutte_embeddings (instance) : 
    
    possible_outer_faces = detect_cycle(instance)

    for outer_face in possible_outer_faces :
        print(outer_face)
        tutte_instance = tutte_embedding(instance,outer_face)
        if is_valid(tutte_instance,False,False) :
            print(f'ratio : {get_polyline_edge_length_ratio (tutte_instance)}')

            
#Choose the valid instance with minimum ratio among all possible outer faces
#Returns empty instance if not found
def best_tutte_embedding (instance) : 
    
    possible_outer_faces = detect_cycle(instance)
    
    best_ratio = 10**9
    best_instance = {}
    best_outer_face = []
    
    for outer_face in possible_outer_faces :
        tutte_instance = tutte_embedding(instance,outer_face)
        if is_valid(tutte_instance,False,False) :
            ratio = get_polyline_edge_length_ratio (tutte_instance)
            if ratio < best_ratio : 
                best_ratio = ratio
                best_instance = copy.deepcopy(tutte_instance)
                best_outer_face = outer_face
    
    return best_instance, best_outer_face

## Networkx planar graph drawing

In [236]:
#Initialize the vertices position of the graph to be planar
def find_planar_drawing(instance) :
    
    #enters the bends as normal nodes in the networkx graph and save the original bend correspondance
    #a bend is represented by the index of the edge, and its order among the other bends of the edge
    
    bend_instance, correspondance = get_bend_correspondance(instance)
    
    G1 = nx.Graph()
    n = len(bend_instance['nodes'])
    nodes = [i for i in range (0,n)]
    G1.add_nodes_from(nodes)
    for edge in bend_instance['edges'] :
        G1.add_edge(edge['source'],edge['target'])

    G2 = nx.algorithms.planarity.check_planarity(G1)
    
    if G2[0] == False :
        
        return instance
    
    else :
    
        positions = nx.algorithms.planar_drawing.combinatorial_embedding_to_pos(G2[1])

        for node in positions : 
            if (node < len(instance['nodes'])) : 
                instance['nodes'][node]['x'] = positions[node][0]
                instance['nodes'][node]['y'] = positions[node][1]
            else : 
                i,j = correspondance[node]
                instance['edges'][i]['bends'][j]['x'] = positions[node][0]
                instance['edges'][i]['bends'][j]['y'] = positions[node][1]
        
        return instance

## Random minimization of the ratio

In [287]:
def one_switch(instance):
    n = len(instance['nodes'])
    score = get_polyline_edge_length_ratio(instance)
    node = rd.randint(0, n-1)
    delta = rd.choice([-1, 1])
    coord = rd.choice(['x', 'y'])
    instance['nodes'][node][coord] += delta
    if not(is_valid(instance) and get_polyline_edge_length_ratio(instance) < score):
        instance['nodes'][node][coord] -= delta
    return instance


def all_switch(instance):
    n = len(instance['nodes'])
    score = get_polyline_edge_length_ratio(
        instance) if is_valid(instance) else 10e10
    new_instance = copy.deepcopy(instance)
    for node in range(n):
        delta = rd.choice([-1, 0, 1])
        coord = rd.choice(['x', 'y'])
        new_instance['nodes'][node][coord] += delta
    if (is_valid(new_instance)):
        if get_polyline_edge_length_ratio(new_instance) < score:
            return new_instance
    return instance


def iterate(instance, function, nmax=500):
    for i in range(nmax):
        old_instance = copy.deepcopy(instance)
        instance = function(instance)
        old_ratio = get_polyline_edge_length_ratio(old_instance)
        ratio = get_polyline_edge_length_ratio(instance)
        if ((i*100)/nmax % 10 == 0):
            print((i*100)/nmax, '% ', get_polyline_edge_length_ratio(instance))
    return instance


def expand_instance(instance, nmax=1000):
    n = len(instance['nodes'])
    for i in range(nmax):
        node = rd.randint(0, n-1)
        delta = rd.choice([-1, 1])
        coord = rd.choice(['x', 'y'])
        instance['nodes'][node][coord] += delta
        if (is_valid(instance, True)):
            return


## Spring forces

In [238]:
def apply_force_step(instance, dt=0.1, convex_hull = None, outer_force = 1, integer_coordinates = False):
    
    bend_instance, correspondance = get_bend_correspondance(instance)
    
    n = len(bend_instance["nodes"])
    El = get_edges_length(bend_instance)
    min_length = min(El)
    med_length = sum(El)/len(El)
    displacement = {}
    if convex_hull == None :
        convex_hull = get_convex_hull(bend_instance)
        
    for node in range(n):
        displacement[node] = {'x': 0, 'y': 0}

    for edge in bend_instance["edges"]:
        n1 = edge["source"]
        n2 = edge["target"]
        vec = complex(bend_instance["nodes"][n1]['x'] - bend_instance["nodes"][n2]['x'], bend_instance["nodes"][n1]['y'] - bend_instance["nodes"][n2]['y'])
        vec *= dt
        
        if (n1 in convex_hull) and (n2 not in convex_hull) :
            displacement[n2]['x'] += outer_force*vec.real
            displacement[n2]['y'] += outer_force*vec.imag
        elif (n2 in convex_hull) and (n1 not in convex_hull) :
            displacement[n1]['x'] -= outer_force*vec.real
            displacement[n1]['y'] -= outer_force*vec.imag
        else : 
            if not(n1 in convex_hull):
                displacement[n1]['x'] -= vec.real
                displacement[n1]['y'] -= vec.imag
            if not(n2 in convex_hull):
                displacement[n2]['x'] += vec.real
                displacement[n2]['y'] += vec.imag
        
    for node in range(n):
        for c in ['x', 'y']:
            if integer_coordinates : 
                bend_instance["nodes"][node][c] += int(displacement[node][c])
            else : 
                bend_instance["nodes"][node][c] += displacement[node][c]
            
    instance = apply_bend_correspondance(instance, bend_instance,correspondance)
            
    return instance

## Handling the bendings to help minimize the ratio

In [353]:
#Inserts a bend to the minimum length edge that can still get one more bend if it exists
def split_min_edge(instance) : 
    
    max_bends = instance['bends']
    
    l_min = sqrt(instance['width']**2+instance['height']**2)
    index = -1
    
    for i in range (len(instance['edges'])) : 
        
        edge = instance['edges'][i]
        l = get_edge_length(edge,instance)
            
        if (l < l_min) and (('bends' in edge and len(edge['bends']) < max_bends) or ('bends' not in edge and max_bends>0)) :
            l_min = l 
            index = i
            
    if index == -1 :
        return instance
    
    edge = instance['edges'][index]
    if 'bends' not in edge :
            edge['bends'] = []
    edge['bends'].append({'x':0,'y':0})
    
    instance['edges'][index] = edge
    
    return instance

## Combining several strategies

In [348]:
#Test splitting the minimum length edge when possible for several times and keeps the case with minimum ratio
#Option 1 = tutte embedding, option 2 = networkx planar drawing
def test_split (instance, option, outer_face) : 
    
    all_instances = []
    all_ratios = []
    n = len(instance['edges'])*instance['bends']
    
    for i in range (n) : 
        
        instance = split_min_edge(instance)
        if option == 1 : 
            instance = tutte_embedding(instance,outer_face)
        if option == 2 : 
            instance = find_planar_drawing(instance)
            
        ratio = get_polyline_edge_length_ratio(instance)
        all_instances.append(copy.deepcopy(instance))
        all_ratios.append(ratio)

    index_min = all_ratios.index(min(all_ratios))
    instance = all_instances[index_min]
    
    return instance, all_ratios[index_min]


#Test applying spring force for several times and keeps the case with minimum ratio
def test_force (instance, outer_face, outer_force, integer_coordinates) : 
    
    all_instances = []
    all_ratios = []
    
    for i in range (10**3) : 
        instance = apply_force_step(instance,0.1,outer_face,outer_force,integer_coordinates)
        ratio = get_polyline_edge_length_ratio(instance)
        all_instances.append(copy.deepcopy(instance))
        all_ratios.append(ratio)

    index_min = all_ratios.index(min(all_ratios))
    instance = all_instances[index_min]
    
    return instance, all_ratios[index_min]


#Rounds the float coordinates to integer coordinates within the maximum size grid
#Does not handle validity problems that can be created
def approximate (instance) : 
    
    for node in instance['nodes'] :
        node['x'] = find_closest_grid_point(node['x'],instance['width'])
        node['y'] = find_closest_grid_point(node['y'],instance['height'])
    for edge in instance['edges'] : 
        if 'bends' in edge :
            for bend in edge['bends'] :
                bend['x'] = find_closest_grid_point(bend['x'],instance['width'])
                bend['y'] = find_closest_grid_point(bend['y'],instance['height'])
                
    return instance


def find_best_tutte_ratio (file_name) : 
    
    instance = read_input_instance(file_name)
    ratio = get_polyline_edge_length_ratio(instance)
    
    tutte_instance = copy.deepcopy(instance)
    t1 = time.time()
    tutte_instance, outer_face = best_tutte_embedding(tutte_instance)
    t2 = time.time()
    print(f'best tutte time : {t2-t1}')
    
    if tutte_instance == {} : 
        tutte_ratio = 10**9
        print('no valid tutte embedding, ratio 10**9')
        
    else : 
        
        tutte_ratio = get_polyline_edge_length_ratio(tutte_instance)
        print(f'initial ratio {tutte_ratio}')
        
        tutte_instance1a = copy.deepcopy(tutte_instance)
        tutte_instance1a, tutte_ratio1a = test_split(tutte_instance1a,1,outer_face)
        t3 = time.time()
        print(f'1a time : {t3-t2}')
        print(f'step 1a, ratio {tutte_ratio1a}')
        
        tutte_instance1b = copy.deepcopy(tutte_instance1a)
        tutte_instance1b, tutte_ratio1b = test_force(tutte_instance1b,outer_face,2,False)
        t4 = time.time()
        print(f'1b time : {t4-t3}')
        print(f'step 1b, ratio {tutte_ratio1b}')
        
        tutte_instance2a = copy.deepcopy(tutte_instance)
        tutte_instance2a, tutte_ratio2a = test_force(tutte_instance2a,outer_face,2,False)
        t5 = time.time()
        print(f'2a time : {t5-t4}')
        print(f'step 2a, ratio {tutte_ratio2a}')
        
        tutte_instance2b = copy.deepcopy(tutte_instance2a)
        tutte_instance2b, tutte_ratio2b = test_split(tutte_instance2a,1,outer_face)
        t6 = time.time()
        print(f'2b time : {t6-t5}')
        print(f'step 2b, ratio {tutte_ratio2b}')
        
        tutte_instance1a = approximate(tutte_instance1a)
        print('\n random 1a')
        tutte_instance1a = iterate(tutte_instance1a,one_switch)
        t7 = time.time()
        print(f'random 1a time : {t7-t6}')
        tutte_ratio1a = get_polyline_edge_length_ratio(tutte_instance1a)
        print(f'\nproxy + random 1a, ratio {tutte_ratio1a}')
        
        tutte_instance1b = approximate(tutte_instance1b)
        print('\n random 1b')
        tutte_instance1b= iterate(tutte_instance1b,one_switch)
        t8 = time.time()
        print(f'random 1b time : {t8-t7}')
        tutte_ratio1b = get_polyline_edge_length_ratio(tutte_instance1b)
        print(f'\nproxy + random 1b, ratio {tutte_ratio1b}')
        
        tutte_instance2a = approximate(tutte_instance2a)
        print('\n random 2a')
        tutte_instance2a = iterate(tutte_instance2a,one_switch)
        t9 = time.time()
        print(f'random 2a time : {t9-t8}')
        tutte_ratio2a = get_polyline_edge_length_ratio(tutte_instance2a)
        print(f'\nproxy + random 2a, ratio {tutte_ratio2a}')
        
        tutte_instance2b = approximate(tutte_instance2b)
        print('\n random2b')
        tutte_instance2b = iterate(tutte_instance2b,one_switch)
        t10 = time.time()
        print(f'random 2b time : {t10-t9}')
        tutte_ratio2b = get_polyline_edge_length_ratio(tutte_instance2b)
        print(f'\nproxy + random 2b, ratio {tutte_ratio2b}')
        
        if (is_valid(tutte_instance1a)) and (is_valid(tutte_instance1b)) :
            
            if (tutte_ratio1a <= tutte_ratio1b) : 
                tutte_instance1 = tutte_instance1a
                tutte_ratio1 = tutte_ratio1a
                print('ratio1 = 1a')
                
            else :
                tutte_instance1 = tutte_instance1b
                tutte_ratio1 = tutte_ratio1b
                print('ratio1 = 1b')
                
        elif is_valid(tutte_instance1a) : 
            tutte_instance1 = tutte_instance1a
            tutte_ratio1 = tutte_ratio1a
            print('ratio1 = 1a')
            
        elif is_valid(tutte_instance1b) : 
            tutte_instance1 = tutte_instance1b
            tutte_ratio1 = tutte_ratio1b
            print('ratio1 = 1b')
            
        else :
            tutte_instance1 = tutte_instance1a
            tutte_ratio1 = 10**9
            print('ratio1 = 10**9, no valid instance')
            
            
        if (is_valid(tutte_instance2a)) and (is_valid(tutte_instance2b)) :
            
            if (tutte_ratio2a <= tutte_ratio2b) : 
                tutte_instance2 = tutte_instance2a
                tutte_ratio2 = tutte_ratio2a
                print('ratio2 = 2a')
                
            else :
                tutte_instance2 = tutte_instance2b
                tutte_ratio2 = tutte_ratio2b
                print('ratio2 = 2b')
                
        elif is_valid(tutte_instance2a) : 
            tutte_instance2 = tutte_instance2a
            tutte_ratio2 = tutte_ratio2a
            print('ratio2 = 2a')
            
        elif is_valid(tutte_instance2b) : 
            tutte_instance2 = tutte_instance2b
            tutte_ratio2 = tutte_ratio2b
            print('ratio2 = 2b')
            
        else :
            tutte_instance2 = tutte_instance2a
            tutte_ratio2 = 10**9
            print('ratio2 = 10**9, no valid instance')
            
            
        if (is_valid(tutte_instance1)) and (is_valid(tutte_instance2)) :
            
            if (tutte_ratio1 <= tutte_ratio2) : 
                if tutte_ratio1 < tutte_ratio : 
                    tutte_instance = tutte_instance1
                    tutte_ratio = tutte_ratio1
                    print('ratio = ratio1')
                
            else :
                if tutte_ratio2 < tutte_ratio : 
                    tutte_instance = tutte_instance2
                    tutte_ratio = tutte_ratio2
                    print('ratio = ratio2')
                
        elif is_valid(tutte_instance1) : 
            if tutte_ratio1 < tutte_ratio : 
                tutte_instance = tutte_instance1
                tutte_ratio = tutte_ratio1
                print('ratio = ratio1')
            
        elif is_valid(tutte_instance2) : 
            if tutte_ratio2 < tutte_ratio : 
                tutte_instance = tutte_instance2
                tutte_ratio = tutte_ratio2
                print('ratio = ratio2')
        
    write_instance(tutte_instance,'tutte_'+file_name)       
    return tutte_instance, tutte_ratio


def find_best_nx_ratio (file_name) : 
    
    instance = read_input_instance(file_name)
    ratio = get_polyline_edge_length_ratio(instance)
    
    nx_instance = copy.deepcopy(instance)
    t1 = time.time()
    nx_instance = find_planar_drawing (nx_instance)
    t2 = time.time()
    print(f'nx drawing time {t2-t1}')
    outer_face = get_convex_hull(nx_instance)   
    nx_ratio = get_polyline_edge_length_ratio(nx_instance)
    print(f'initial ratio {nx_ratio}')

    nx_instance1a = copy.deepcopy(nx_instance)
    nx_instance1a, nx_ratio1a = test_split(nx_instance1a,2,outer_face)
    t3 = time.time()
    print(f'1a time {t3-t2}')
    print(f'step 1a, ratio {nx_ratio1a}')

    nx_instance1b = copy.deepcopy(nx_instance1a)
    nx_instance1b, nx_ratio1b = test_force(nx_instance1b,outer_face,1,False)
    t4 = time.time()
    print(f'1b time {t4-t3}')
    print(f'step 1b, ratio {nx_ratio1b}')

    nx_instance2a = copy.deepcopy(nx_instance)
    nx_instance2a, nx_ratio2a = test_force(nx_instance2a,outer_face,1,False)
    t5 = time.time()
    print(f'2a time {t5-t4}')
    print(f'step 2a, ratio {nx_ratio2a}')

    nx_instance2b = copy.deepcopy(nx_instance2a)
    nx_instance2b, nx_ratio2b = test_split(nx_instance2a,2,outer_face)
    t6 = time.time()
    print(f'2b time {t6-t5}')
    print(f'step 2b, ratio {nx_ratio2b}')

    nx_instance1a = approximate(nx_instance1a)
    print('\n random 1a')
    nx_instance1a = iterate(nx_instance1a,one_switch)
    t7 = time.time()
    print(f'random 1a time {t7-t6}')
    nx_ratio1a = get_polyline_edge_length_ratio(nx_instance1a)
    print(f'\nproxy + random 1a, ratio {nx_ratio1a}')

    nx_instance1b = approximate(nx_instance1b)
    print('\n random 1b')
    nx_instance1b= iterate(nx_instance1b,one_switch)
    t8 = time.time()
    print(f'random 1b time {t8-t7}')
    nx_ratio1b = get_polyline_edge_length_ratio(nx_instance1b)
    print(f'\nproxy + random 1b, ratio {nx_ratio1b}')

    nx_instance2a = approximate(nx_instance2a)
    print('\n random 2a')
    nx_instance2a = iterate(nx_instance2a,one_switch)
    t9 = time.time()
    print(f'random 2a time {t9-t8}')
    nx_ratio2a = get_polyline_edge_length_ratio(nx_instance2a)
    print(f'\nproxy + random 2a, ratio {nx_ratio2a}')

    nx_instance2b = approximate(nx_instance2b)
    print('\n random2b')
    nx_instance2b = iterate(nx_instance2b,one_switch)
    t10 = time.time()
    print(f'random 2b time {t10-t9}')
    nx_ratio2b = get_polyline_edge_length_ratio(nx_instance2b)
    print(f'\nproxy + random 2b, ratio {nx_ratio2b}')

    if (is_valid(nx_instance1a)) and (is_valid(nx_instance1b)) :

        if (nx_ratio1a <= nx_ratio1b) : 
            nx_instance1 = nx_instance1a
            nx_ratio1 = nx_ratio1a
            print('ratio1 = 1a')

        else :
            nx_instance1 = nx_instance1b
            nx_ratio1 = nx_ratio1b
            print('ratio1 = 1b')

    elif is_valid(nx_instance1a) : 
        nx_instance1 = nx_instance1a
        nx_ratio1 = nx_ratio1a
        print('ratio1 = 1a')

    elif is_valid(nx_instance1b) : 
        nx_instance1 = nx_instance1b
        nx_ratio1 = nx_ratio1b
        print('ratio1 = 1b')

    else :
        nx_instance1 = nx_instance1a
        nx_ratio1 = 10**9
        print('ratio1 = 10**9, no valid instance')


    if (is_valid(nx_instance2a)) and (is_valid(nx_instance2b)) :

        if (nx_ratio2a <= nx_ratio2b) : 
            nx_instance2 = nx_instance2a
            nx_ratio2 = nx_ratio2a
            print('ratio2 = 2a')

        else :
            nx_instance2 = nx_instance2b
            nx_ratio2 = nx_ratio2b
            print('ratio2 = 2b')

    elif is_valid(nx_instance2a) : 
        nx_instance2 = nx_instance2a
        nx_ratio2 = nx_ratio2a
        print('ratio2 = 2a')

    elif is_valid(nx_instance2b) : 
        nx_instance2 = nx_instance2b
        nx_ratio2 = nx_ratio2b
        print('ratio2 = 2b')

    else :
        nx_instance2 = nx_instance2a
        nx_ratio2 = 10**9
        print('ratio2 = 10**9, no valid instance')


    if (is_valid(nx_instance1)) and (is_valid(nx_instance2)) :

        if (nx_ratio1 <= nx_ratio2) : 
            if nx_ratio1 < nx_ratio : 
                nx_instance = nx_instance1
                nx_ratio = nx_ratio1
                print('ratio = ratio1')

        else :
            if nx_ratio2 < nx_ratio : 
                nx_instance = nx_instance2
                nx_ratio = nx_ratio2
                print('ratio = ratio2')

    elif is_valid(nx_instance1) : 
        if nx_ratio1 < nx_ratio : 
            nx_instance = nx_instance1
            nx_ratio = nx_ratio1
            print('ratio = ratio1')

    elif is_valid(nx_instance2) : 
        if nx_ratio2 < nx_ratio : 
            nx_instance = nx_instance2
            nx_ratio = nx_ratio2
            print('ratio = ratio2')
        
    write_instance(nx_instance,'nx_'+file_name)       
    return nx_instance, nx_ratio


def find_best_ratio (file_name) : 
    
    instance = read_input_instance(file_name) 
    ratio = get_polyline_edge_length_ratio(instance)
    
    print('Tutte \n')
    tutte_instance, tutte_ratio = find_best_tutte_ratio(file_name)
    print('\n\n Networkx \n')
    nx_instance, nx_ratio = find_best_nx_ratio(file_name)
    
    
    print('\nSynthesis\n')
    if not is_valid(instance) : 
        instance = {}
        ratio = 10**9
    
    if (is_valid(tutte_instance) and is_valid(nx_instance)) : 
        
        if (tutte_ratio < nx_ratio) : 
            if (tutte_ratio < ratio) :
                instance = tutte_instance
                ratio = tutte_ratio
                print('tutte')
                
        else :
            if (nx_ratio < ratio) : 
                instance = nx_instance
                ratio = nx_ratio
                print('nx')
                
    elif is_valid(tutte_instance) : 
        if (tutte_ratio < ratio) : 
            instance = tutte_instance
            ratio = tutte_ratio
            print('tutte')
            
    elif is_valid(nx_instance) : 
        if (nx_ratio < ratio) : 
            instance = nx_instance
            ratio = nx_ratio
            print('nx')
            
    
    write_instance(instance,file_name)
    return instance, ratio
        

In [291]:
def find_all_inputs () : 
    inputs = [file_name[:-5] for file_name in listdir('datasets/json/')]
    inputs.remove('.DS_')
    for input_name in inputs : 
        print('\n' +input_name + '\n')
        find_best_ratio(input_name)

In [349]:
find_all_inputs()


delaunay80

Tutte 

best tutte time : 1.4783108234405518
initial ratio 37.43660406471932
1a time : 18.451144218444824
step 1a, ratio 27.792029704763497
1b time : 6.791070938110352
step 1b, ratio 23.341036088819017
2a time : 3.155674934387207
step 2a, ratio 27.792029704809934
2b time : 17.176337242126465
step 2b, ratio 27.792029704763497

 random 1a
0.0 %  26.70804283377514
10.0 %  26.70804283377514
20.0 %  26.70804283377514
30.0 %  26.70804283377514
40.0 %  26.70804283377514
50.0 %  26.70804283377514
60.0 %  26.70804283377514
70.0 %  26.70804283377514
80.0 %  26.70804283377514
90.0 %  26.70804283377514
random 1a time : 112.95992088317871

proxy + random 1a, ratio 26.70804283377514

 random 1b
0.0 %  26.70804283377514
10.0 %  26.70804283377514
20.0 %  26.70804283377514
30.0 %  22.627416997969522
40.0 %  21.916117736711854
50.0 %  21.916117736711854
60.0 %  21.916117736711854
70.0 %  21.607260539018117
80.0 %  21.607260539018117
90.0 %  21.607260539018117
random 1b time : 113.4898710250

1b time 17.61405086517334
step 1b, ratio 609.5260453828039
2a time 13.048204898834229
step 2a, ratio 594.0
2b time 166.17929410934448
step 2b, ratio 381.0259833659641

 random 1a
0.0 %  385.4981193209637
10.0 %  385.4981193209637
20.0 %  385.4981193209637
30.0 %  385.4981193209637
40.0 %  385.4981193209637
50.0 %  385.4981193209637
60.0 %  385.4981193209637
70.0 %  385.4981193209637
80.0 %  385.4981193209637
90.0 %  385.4981193209637
random 1a time 5.97536826133728

proxy + random 1a, ratio 385.4981193209637

 random 1b
0.0 %  609.5260453828039
10.0 %  609.5260453828039
20.0 %  609.5260453828039
30.0 %  609.5260453828039
40.0 %  609.5260453828039
50.0 %  609.5260453828039
60.0 %  609.5260453828039
70.0 %  609.5260453828039
80.0 %  609.5260453828039
90.0 %  609.5260453828039
random 1b time 5.95056676864624

proxy + random 1b, ratio 609.5260453828039

 random 2a
0.0 %  843.580318006964
10.0 %  843.580318006964
20.0 %  843.580318006964
30.0 %  843.580318006964
40.0 %  843.580318006964
50.

1a time 0.7093102931976318
step 1a, ratio 32.707106781186546
1b time 1.3708019256591797
step 1b, ratio 31.23606797749979
2a time 1.0548229217529297
step 2a, ratio 40.0
2b time 0.7944791316986084
step 2b, ratio 30.781008367064572

 random 1a
0.0 %  32.707106781186546
10.0 %  32.707106781186546
20.0 %  32.707106781186546
30.0 %  32.707106781186546
40.0 %  32.707106781186546
50.0 %  32.707106781186546
60.0 %  32.707106781186546
70.0 %  32.707106781186546
80.0 %  32.707106781186546
90.0 %  32.707106781186546
random 1a time 0.49353694915771484

proxy + random 1a, ratio 32.707106781186546

 random 1b
0.0 %  31.23606797749979
10.0 %  31.23606797749979
20.0 %  31.23606797749979
30.0 %  31.23606797749979
40.0 %  31.23606797749979
50.0 %  31.23606797749979
60.0 %  31.23606797749979
70.0 %  31.23606797749979
80.0 %  31.23606797749979
90.0 %  31.23606797749979
random 1b time 0.5837631225585938

proxy + random 1b, ratio 31.23606797749979

 random 2a
0.0 %  53.533008588991066
10.0 %  53.533008588991

random 2b time 0.17819809913635254

proxy + random 2b, ratio 6.798373876248843
ratio1 = 1a
ratio2 = 10**9, no valid instance
ratio = ratio1

Synthesis

tutte

sphere

Tutte 

best tutte time : 0.36666011810302734
initial ratio 50.419043534292655
1a time : 11.449283838272095
step 1a, ratio 33.32217503950367
1b time : 4.249810218811035
step 1b, ratio 26.523694821981945
2a time : 1.5644769668579102
step 2a, ratio 36.92613545859414
2b time : 11.261960744857788
step 2b, ratio 33.32217503950367

 random 1a
0.0 %  34.666666666666664
10.0 %  30.460894756599057
20.0 %  30.460894756599057
30.0 %  29.899506124850568
40.0 %  29.757143834462894
50.0 %  29.757143834462894
60.0 %  29.757143834462894
70.0 %  29.479087726056708
80.0 %  29.302499111981756
90.0 %  29.302499111981756
random 1a time : 41.64278817176819

proxy + random 1a, ratio 28.935715495535828

 random 1b
0.0 %  27.165202996229112
10.0 %  26.0
20.0 %  24.53744657394262
30.0 %  24.53744657394262
40.0 %  24.499433100017274
50.0 %  22.3518

## Graph visualization

In [253]:
#Draw instance
def draw_instance (instance, show_max_grid = False) : 
        

    window = Tk()

    node = get_nodes(instance)
    bend = get_bends(instance)
    vertices = node+bend

    if show_max_grid : 
        
        instance_xmin, instance_xmax = floor(min(vertices, key = lambda t:t[0])[0]), ceil(max(vertices, key = lambda t:t[0])[0])
        instance_ymin, instance_ymax = floor(min(vertices, key = lambda t:t[1])[1]), ceil(max(vertices, key = lambda t:t[1])[1])
        xmin = 0
        xmax = instance['width']
        ymin = 0
        ymax = instance['height']

        #translate points to get in the grid
        if (instance_xmin < xmin) : 
            for i in range (len(instance['nodes'])) :
                instance['nodes'][i]['x'] += xmin - instance_xmin
        if (instance_ymin < ymin) : 
            for i in range (len(instance['nodes'])) :
                instance['nodes'][i]['y'] += ymin - instance_ymin
        if (instance_xmax > xmax) : 
            for i in range (len(instance['nodes'])) :
                instance['nodes'][i]['x'] -= instance_xmax - xmax         
        if (instance_ymax > ymax) : 
            for i in range (len(instance['nodes'])) :
                instance['nodes'][i]['y'] -= instance_ymax - ymax

        node = get_nodes(instance)
        bend = get_bends(instance)
        vertices = node+bend
        
    else : 
        
        xmin, xmax = floor(min(vertices, key = lambda t:t[0])[0]), ceil(max(vertices, key = lambda t:t[0])[0])
        ymin, ymax = floor(min(vertices, key = lambda t:t[1])[1]), ceil(max(vertices, key = lambda t:t[1])[1])
        
        
    #canvas

    x_nb_of_boxes = xmax - xmin + 2
    y_nb_of_boxes = ymax - ymin + 2

    x_size_max = 1400
    y_size_max = 800

    x_step_max = int(x_size_max/x_nb_of_boxes)
    y_step_max = int(y_size_max/y_nb_of_boxes)

    step = min(x_step_max,y_step_max)
    x_size = step*x_nb_of_boxes
    y_size = step*y_nb_of_boxes
    marge = 5

    canvas = Canvas(window, bg="light gray", height=y_size+2*marge, width=x_size+2*marge)
    canvas.pack()
        

    #grid

    x = 0
    while (x <= x_nb_of_boxes) :
        canvas.create_line(marge+step*x, marge, marge+step*x, marge+y_size, fill='black')
        x += 1

    y = 0
    while (y <= y_nb_of_boxes) :
        canvas.create_line(marge, marge+step*y, marge+x_size, marge+step*y, fill='black')
        y += 1

    #points

    radius = 5 #nodes
    for point in node :
        i = point[0]
        j = ymax - point[1] + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_oval(x-radius,y+radius,x+radius,y-radius, fill='red')

    radius = 2 #bends
    for point in bend :
        i = point[0]
        j = ymax - point[1] + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_oval(x-radius,y+radius,x+radius,y-radius, fill='black')

    #edges

    for edge in instance['edges'] :

        if 'bends' in edge :

            n = len(edge['bends'])

            i1,j1 = node[edge['source']]
            i2,j2 = edge['bends'][0]['x'],edge['bends'][0]['y']
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

            i1,j1 = node[edge['target']]
            i2,j2 = edge['bends'][-1]['x'],edge['bends'][-1]['y']
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

            for i in range (1,n): 

                i1,j1 = edge['bends'][i-1]['x'],edge['bends'][i-1]['y']
                i2,j2 = edge['bends'][i]['x'],edge['bends'][i]['y']
                j1 = ymax - j1 + ymin
                j2 = ymax - j2 + ymin

                x1 = marge + step*(i1 + 1 - xmin)
                x2 = marge + step*(i2 + 1 - xmin)
                y1 = marge + step*(j1 + 1 - ymin)
                y2 = marge + step*(j2 + 1 - ymin)

                canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

        else :

            i1,j1 = node[edge['source']]
            i2,j2 = node[edge['target']]
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

    #node ids

    radius = 5
    for k in range (len(node)) :
        i,j = node[k]
        j = ymax - j + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_text(x+radius+3,y+radius+3,text=str(k))
        
        
    window.mainloop()
    

In [244]:
#Visualize the json file
def draw_input_file (instance_name, show_max_grid = False) : 
    
    instance = read_input_instance(instance_name)
    if not is_valid(instance) : 
        instance = find_planar_drawing(instance)
        
    draw_instance(instance, show_max_grid)
    
    
#Visualize the json file
def draw_output_file (instance_name, show_max_grid = False) : 
    
    instance = read_output_instance(instance_name)
    if not is_valid(instance) : 
        instance = find_planar_drawing(instance)
        
    draw_instance(instance, show_max_grid)

In [354]:
#Shows one tutte iteration at a time by pressing any key
def draw_tutte_step (event) : 
    
    global instance, window, canvas, outer_face
    
    instance = tutte_step(outer_face,instance)
    #print(f'Valid : {is_valid(instance,True)}')
    if is_valid(instance,False,False) : 
        print(get_polyline_edge_length_ratio (instance))

    node = get_nodes(instance)
    bend = get_bends(instance)
    vertices = node+bend  
        
    xmin, xmax = floor(min(vertices, key = lambda t:t[0])[0]), ceil(max(vertices, key = lambda t:t[0])[0])
    ymin, ymax = floor(min(vertices, key = lambda t:t[1])[1]), ceil(max(vertices, key = lambda t:t[1])[1])
        
        
    #canvas

    x_nb_of_boxes = xmax - xmin + 2
    y_nb_of_boxes = ymax - ymin + 2

    x_size_max = 1400
    y_size_max = 800

    x_step_max = int(x_size_max/x_nb_of_boxes)
    y_step_max = int(y_size_max/y_nb_of_boxes)

    step = min(x_step_max,y_step_max)
    x_size = step*x_nb_of_boxes
    y_size = step*y_nb_of_boxes
    marge = 5

    canvas.pack_forget()
    canvas = Canvas(window, bg="light gray", height=y_size+2*marge, width=x_size+2*marge)
    canvas.pack()
        

    #grid

    x = 0
    while (x <= x_nb_of_boxes) :
        canvas.create_line(marge+step*x, marge, marge+step*x, marge+y_size, fill='black')
        x += 1

    y = 0
    while (y <= y_nb_of_boxes) :
        canvas.create_line(marge, marge+step*y, marge+x_size, marge+step*y, fill='black')
        y += 1

    #points

    radius = 5 #nodes
    for point in node :
        i = point[0]
        j = ymax - point[1] + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_oval(x-radius,y+radius,x+radius,y-radius, fill='red')

    radius = 2 #bends
    for point in bend :
        i = point[0]
        j = ymax - point[1] + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_oval(x-radius,y+radius,x+radius,y-radius, fill='black')

    #edges

    for edge in instance['edges'] :

        if 'bends' in edge :

            n = len(edge['bends'])

            i1,j1 = node[edge['source']]
            i2,j2 = edge['bends'][0]['x'],edge['bends'][0]['y']
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

            i1,j1 = node[edge['target']]
            i2,j2 = edge['bends'][-1]['x'],edge['bends'][-1]['y']
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

            for i in range (1,n): 

                i1,j1 = edge['bends'][i-1]['x'],edge['bends'][i-1]['y']
                i2,j2 = edge['bends'][i]['x'],edge['bends'][i]['y']
                j1 = ymax - j1 + ymin
                j2 = ymax - j2 + ymin

                x1 = marge + step*(i1 + 1 - xmin)
                x2 = marge + step*(i2 + 1 - xmin)
                y1 = marge + step*(j1 + 1 - ymin)
                y2 = marge + step*(j2 + 1 - ymin)

                canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

        else :

            i1,j1 = node[edge['source']]
            i2,j2 = node[edge['target']]
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

    #node ids

    radius = 5
    for k in range (len(node)) :
        i,j = node[k]
        j = ymax - j + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_text(x+radius+3,y+radius+3,text=str(k))
        

instance = read_input_instance('small')
outer_face, instance = tutte_initialization(instance)
window = Tk()
canvas = Canvas(window, bg="light gray",height = 100, width = 100)
window.bind ("<Any-KeyPress>", draw_tutte_step)
window.mainloop()

10.606601717798215
11.778165511818942
11.779116674387181
11.696313133814993
11.65517042489617
11.637159055604705
11.629402142423963
11.626068932519022
11.624636850989639
11.62402154076895
11.62375715482689
11.62364355137321
11.623594736881355
11.62357376160029
11.623564748637557
11.623560875814182
11.623559211681494


In [347]:
#Shows one force spring iteration at a time by pressing any key
def draw_force_step (event) : 
    
    global instance, window, canvas, convex_hull
    
    if is_valid(instance,False,False) : 
        print(get_polyline_edge_length_ratio (instance))

    node = get_nodes(instance)
    bend = get_bends(instance)
    vertices = node+bend  
        
    xmin, xmax = floor(min(vertices, key = lambda t:t[0])[0]), ceil(max(vertices, key = lambda t:t[0])[0])
    ymin, ymax = floor(min(vertices, key = lambda t:t[1])[1]), ceil(max(vertices, key = lambda t:t[1])[1])
        
        
    #canvas

    x_nb_of_boxes = xmax - xmin + 2
    y_nb_of_boxes = ymax - ymin + 2

    x_size_max = 1400
    y_size_max = 800

    x_step_max = int(x_size_max/x_nb_of_boxes)
    y_step_max = int(y_size_max/y_nb_of_boxes)

    step = min(x_step_max,y_step_max)
    x_size = step*x_nb_of_boxes
    y_size = step*y_nb_of_boxes
    marge = 5

    canvas.pack_forget()
    canvas = Canvas(window, bg="light gray", height=y_size+2*marge, width=x_size+2*marge)
    canvas.pack()
        

    #grid

    x = 0
    while (x <= x_nb_of_boxes) :
        canvas.create_line(marge+step*x, marge, marge+step*x, marge+y_size, fill='black')
        x += 1

    y = 0
    while (y <= y_nb_of_boxes) :
        canvas.create_line(marge, marge+step*y, marge+x_size, marge+step*y, fill='black')
        y += 1

    #points

    radius = 5 #nodes
    for point in node :
        i = point[0]
        j = ymax - point[1] + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_oval(x-radius,y+radius,x+radius,y-radius, fill='red')

    radius = 2 #bends
    for point in bend :
        i = point[0]
        j = ymax - point[1] + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_oval(x-radius,y+radius,x+radius,y-radius, fill='black')

    #edges

    for edge in instance['edges'] :

        if 'bends' in edge :

            n = len(edge['bends'])

            i1,j1 = node[edge['source']]
            i2,j2 = edge['bends'][0]['x'],edge['bends'][0]['y']
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

            i1,j1 = node[edge['target']]
            i2,j2 = edge['bends'][-1]['x'],edge['bends'][-1]['y']
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

            for i in range (1,n): 

                i1,j1 = edge['bends'][i-1]['x'],edge['bends'][i-1]['y']
                i2,j2 = edge['bends'][i]['x'],edge['bends'][i]['y']
                j1 = ymax - j1 + ymin
                j2 = ymax - j2 + ymin

                x1 = marge + step*(i1 + 1 - xmin)
                x2 = marge + step*(i2 + 1 - xmin)
                y1 = marge + step*(j1 + 1 - ymin)
                y2 = marge + step*(j2 + 1 - ymin)

                canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

        else :

            i1,j1 = node[edge['source']]
            i2,j2 = node[edge['target']]
            j1 = ymax - j1 + ymin
            j2 = ymax - j2 + ymin

            x1 = marge + step*(i1 + 1 - xmin)
            x2 = marge + step*(i2 + 1 - xmin)
            y1 = marge + step*(j1 + 1 - ymin)
            y2 = marge + step*(j2 + 1 - ymin)

            canvas.create_line(x1,y1,x2,y2,fill='blue', width=2)

    #node ids

    radius = 5
    for k in range (len(node)) :
        i,j = node[k]
        j = ymax - j + ymin
        x = marge + step*(i + 1 - xmin)
        y = marge + step*(j + 1 - ymin)
        canvas.create_text(x+radius+3,y+radius+3,text=str(k))
        
    instance = apply_force_step(instance,0.1,convex_hull,2)
        

instance, convex_hull = best_tutte_embedding(read_input_instance('delaunay80'))
print(convex_hull)

window = Tk()
canvas = Canvas(window, bg="light gray",height = 100, width = 100)
window.bind ("<Any-KeyPress>", draw_force_step)
window.mainloop()

[1, 2, 3, 0]
37.43660406471932
60.096530552613444
37.4366040647193
60.09653055261344
37.43660406471932
59.65306351918872
36.39784129860749
59.1652909907914
35.73503306940158
58.72315689017469
35.11019192755545
58.30798952527442
34.522313190397035
57.92947562538496
33.993762318593525
57.58403040956951
33.5257239139632
57.26940156421475
33.11333909855381
56.98251381152214
32.74934964035533
56.72055412231729
32.4264291860733
56.480941272053336
32.137971022601455
56.26138880711423
31.87835786978321
56.059883006555076
31.642955593857337
55.87465605517956
31.428004145136253
55.704154331856785
31.230476938046372
55.547008944612
31.047943424537316
55.402009550644706
30.878448268119048
55.26808186506706
30.720410844119744
55.14426859643609
30.572544274080407
55.02971339783177
30.43379146226856
54.92364738513185
30.303275172052754
54.8253778062163
30.18025934346959
54.73427849653541
30.06411924010714
54.64978181214148
29.95431844717386
54.571371783046345
29.85039114536448
54.49857827506488
29.75